In [1]:
import requests 
import datetime
import pandas as pd
import altair as alt


from dotenv import load_dotenv
import os

load_dotenv()

# api_key=os.getenv("WEATHERAPI_APIKEY")

True

In [2]:
# Historical data does not work with the free key
# response = requests.get(f"https://api.weatherapi.com/v1/current.json?key={api_key}&q=Berlin&dt=2015-01-01")
# berlin = response.json()

In [3]:
# I thought about doing an API-Call, but downloading the csv was just so much easier. 

# response = requests.get('https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.405&start_date=2015-06-16&end_date=2025-06-15&daily=temperature_2m_mean,rain_sum,snowfall_sum,sunshine_duration')

# berlin = response.json()

# response = requests.get('https://archive-api.open-meteo.com/v1/archive?latitude=40.6782&longitude=73.9442&start_date=2015-06-16&end_date=2025-06-15&daily=temperature_2m_mean,rain_sum,snowfall_sum,sunshine_duration')

# new_york = response.json()

Data source: https://open-meteo.com/

This describtion works for both dataframes - one will have berlin added, the other one will have brooklyn added

day: date of the measures
temperature: mean temperature on that day in Celcius
rain: sum of rain on that day in millimeter
snowfall: sum of snowfall on that day in Centimeter
sunshine: duration of sunshine in that day in hours (converted von seconds to hours)

In [4]:
df = pd.read_csv('raw/weather_ berlin.csv', header=2)

In [5]:
# rename columns

(
    df
    .rename(
        columns={
            'time': 'day', 
            'temperature_2m_mean (°C)' : 'temperature', 
            'rain_sum (mm)' : 'rain', 
            'snowfall_sum (cm)': 'snowfall', 
            'sunshine_duration (s)':'sunshine'
        }, 
        inplace=True
    )
)

In [6]:
df

,day,temperature,rain,snowfall,sunshine
0,2015-06-16,12.3,0.0,0.0,17109.14
1,2015-06-17,15.2,0.5,0.0,46747.82
2,2015-06-18,15.9,4.2,0.0,30454.32
3,2015-06-19,14.3,0.1,0.0,52821.24
4,2015-06-20,13.8,3.1,0.0,46659.14
...,...,...,...,...,...
3648,2025-06-11,15.8,0.9,0.0,48035.18
3649,2025-06-12,16.8,0.0,0.0,55367.75
3650,2025-06-13,19.1,0.0,0.0,55407.55
3651,2025-06-14,23.4,0.0,0.0,55486.68


In [7]:
# calculate sunshine from seconds to hours

df['sunshine_hours'] = df['sunshine'] / 3600
df = df.drop('sunshine', axis=1)

In [8]:
# add month to data
df['day'] = pd.to_datetime(df['day'])

In [9]:
df['month_year'] = df['day'].dt.to_period('M')
df['year'] = df['day'].dt.to_period('Y')

In [10]:
# add season according to month
# Dez, Jan, Feb == Winter
# Mar, Apr, Mai == Spring
# Jun, Jul, Aug == Summer
# Sep, Oct, Nov == Fall

df['month_num'] = df['day'].dt.month

def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'autumn'

df['season'] = df['month_num'].apply(get_season)

In [11]:
df

,day,temperature,rain,snowfall,sunshine_hours,month_year,year,month_num,season
0,2015-06-16,12.3,0.0,0.0,4.752539,2015-06,2015,6,summer
1,2015-06-17,15.2,0.5,0.0,12.985506,2015-06,2015,6,summer
2,2015-06-18,15.9,4.2,0.0,8.459533,2015-06,2015,6,summer
3,2015-06-19,14.3,0.1,0.0,14.672567,2015-06,2015,6,summer
4,2015-06-20,13.8,3.1,0.0,12.960872,2015-06,2015,6,summer
...,...,...,...,...,...,...,...,...,...
3648,2025-06-11,15.8,0.9,0.0,13.343106,2025-06,2025,6,summer
3649,2025-06-12,16.8,0.0,0.0,15.379931,2025-06,2025,6,summer
3650,2025-06-13,19.1,0.0,0.0,15.390986,2025-06,2025,6,summer
3651,2025-06-14,23.4,0.0,0.0,15.412967,2025-06,2025,6,summer


In [12]:
# save cleaned file with calculations

df.to_csv('processed/berlin_cleaned.csv', index=False)

In [13]:
# do the same for the Brooklyn weather to have the same data

df2 = pd.read_csv('raw/weather_brooklyn.csv', header=2)

In [14]:
df2

,time,rain_sum (mm),snowfall_sum (cm),sunshine_duration (s),temperature_2m_mean (°C)
0,2015-06-16,8.9,0.0,18014.44,21.2
1,2015-06-17,0.0,0.0,34487.41,21.6
2,2015-06-18,0.0,0.0,14856.77,18.7
3,2015-06-19,0.3,0.0,38022.73,22.0
4,2015-06-20,0.1,0.0,7130.58,20.5
...,...,...,...,...,...
3648,2025-06-11,0.1,0.0,48881.35,23.2
3649,2025-06-12,0.0,0.0,50199.27,26.7
3650,2025-06-13,0.1,0.0,43295.38,25.0
3651,2025-06-14,4.9,0.0,2993.10,18.8


In [15]:
# rename columns 

(
    df2
    .rename(
        columns={
            'time': 'day', 
            'temperature_2m_mean (°C)' : 'temperature', 
            'rain_sum (mm)' : 'rain', 
            'snowfall_sum (cm)': 'snowfall', 
            'sunshine_duration (s)':'sunshine'
        }, 
        inplace=True
    )
)
df2

,day,rain,snowfall,sunshine,temperature
0,2015-06-16,8.9,0.0,18014.44,21.2
1,2015-06-17,0.0,0.0,34487.41,21.6
2,2015-06-18,0.0,0.0,14856.77,18.7
3,2015-06-19,0.3,0.0,38022.73,22.0
4,2015-06-20,0.1,0.0,7130.58,20.5
...,...,...,...,...,...
3648,2025-06-11,0.1,0.0,48881.35,23.2
3649,2025-06-12,0.0,0.0,50199.27,26.7
3650,2025-06-13,0.1,0.0,43295.38,25.0
3651,2025-06-14,4.9,0.0,2993.10,18.8


In [16]:
# calculate sunshine from seconds to hours
df2['sunshine_hours'] = df2['sunshine'] / 3600
df2 = df2.drop('sunshine', axis=1)

In [17]:
# add month and year to data

df2['day'] = pd.to_datetime(df2['day'])

In [18]:
df2['month_year'] = df2['day'].dt.to_period('M')
df2['year'] = df2['day'].dt.to_period('Y')

In [19]:
# add season according to month
# Dez, Jan, Feb == Winter
# Mar, Apr, Mai == Spring
# Jun, Jul, Aug == Summer
# Sep, Oct, Nov == Fall

df2['month_num'] = df2['day'].dt.month
df2['season'] = df2['month_num'].apply(get_season)
df2

,day,rain,snowfall,temperature,sunshine_hours,month_year,year,month_num,season
0,2015-06-16,8.9,0.0,21.2,5.004011,2015-06,2015,6,summer
1,2015-06-17,0.0,0.0,21.6,9.579836,2015-06,2015,6,summer
2,2015-06-18,0.0,0.0,18.7,4.126881,2015-06,2015,6,summer
3,2015-06-19,0.3,0.0,22.0,10.561869,2015-06,2015,6,summer
4,2015-06-20,0.1,0.0,20.5,1.980717,2015-06,2015,6,summer
...,...,...,...,...,...,...,...,...,...
3648,2025-06-11,0.1,0.0,23.2,13.578153,2025-06,2025,6,summer
3649,2025-06-12,0.0,0.0,26.7,13.944242,2025-06,2025,6,summer
3650,2025-06-13,0.1,0.0,25.0,12.026494,2025-06,2025,6,summer
3651,2025-06-14,4.9,0.0,18.8,0.831417,2025-06,2025,6,summer


In [20]:
df2.describe()

,day,rain,snowfall,temperature,sunshine_hours,month_num
count,3653,3653.000000,3653.000000,3653.000000,3653.000000,3653.000000
mean,2020-06-15 00:00:00,3.275198,0.146573,12.659869,8.284411,6.522310
min,2015-06-16 00:00:00,0.000000,0.000000,-14.900000,0.000000,1.000000
25%,2017-12-15 00:00:00,0.000000,0.000000,5.200000,5.932775,4.000000
50%,2020-06-15 00:00:00,0.000000,0.000000,12.600000,9.072733,7.000000
75%,2022-12-15 00:00:00,2.800000,0.000000,21.000000,11.537692,10.000000
max,2025-06-15 00:00:00,110.500000,28.000000,30.400000,14.000000,12.000000
std,NaN,7.655594,1.267931,9.147561,4.118998,3.449345


In [21]:
# save cleaned file with calculations

df2.to_csv('processed/brooklyn_cleaned.csv', index=False)